# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse  Shipments          Customer  \
0  Jun 22 2022 11:26AM  240182        10  MSP211522  Methapharm, Inc.   
1  Jun 22 2022 11:26AM  240181        10  MSP211523  Methapharm, Inc.   
2  Jun 22 2022 11:26AM  240181        10  MSP211524  Methapharm, Inc.   
3  Jun 22 2022 11:26AM  240178        10  MSP211515  Methapharm, Inc.   
4  Jun 22 2022 11:26AM  240178        10  MSP211516  Methapharm, Inc.   
5  Jun 22 2022 11:26AM  240178        10  MSP211517  Methapharm, Inc.   
6  Jun 22 2022 11:26AM  240178        10  MSP211518  Methapharm, Inc.   
7  Jun 22 2022 11:26AM  240178        10  MSP211519  Methapharm, Inc.   
8  Jun 22 2022 11:26AM  240178        10  MSP211520  Methapharm, Inc.   
9  Jun 22 2022 11:26AM  240178        10  MSP211521  Methapharm, Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
37  240178       Released          7
38  240179       Released          8
39  240180       Released         32
40  240181       Released          2
41  240182       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
240178                NaN        NaN       7.0
240179                NaN        NaN       8.0
240180                NaN        NaN      32.0
240181                NaN        NaN       2.0
240182                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
240085                0.0        8.0       4.0
240086                0.0       16.0       1.0
240089                0.0        0.0       1.0
240092                0.0        1.0       0.0
240102                0.0        1.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
240085                  0          8         4
240086                  0         16         1
240089                  0          0         1
240092                  0          1         0
240102                  0          1         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               240085          0          8         4
1               240086          0         16         1
2               240089          0          0         1
3               240092          0          1         0
4               240102          0          1         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               240085                   8        4
1               240086                  16        1
2               240089                            1
3               240092                   1         
4               240102                   1        1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                  Customer
0   Jun 22 2022 11:26AM  240182        10          Methapharm, Inc.
1   Jun 22 2022 11:26AM  240181        10          Methapharm, Inc.
3   Jun 22 2022 11:26AM  240178        10          Methapharm, Inc.
10  Jun 22 2022 11:22AM  240180        10         ISDIN Corporation
42  Jun 22 2022 11:19AM  240179        10         ISDIN Corporation
50  Jun 22 2022 10:58AM  240177        20  ACI Healthcare USA, Inc.
51  Jun 22 2022 10:49AM  240175        10          Eywa Pharma Inc.
57  Jun 22 2022 10:37AM  240174        19    GCH Granules USA, Inc.
58  Jun 22 2022 10:32AM  240172        10         ISDIN Corporation
96  Jun 22 2022 10:31AM  240173        10         ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                  Customer Completed  \
0  Jun 22 2022 11:26AM  240182        10          Methapharm, Inc.             
1  Jun 22 2022 11:26AM  240181        10          Methapharm, Inc.             
2  Jun 22 2022 11:26AM  240178        10          Methapharm, Inc.             
3  Jun 22 2022 11:22AM  240180        10         ISDIN Corporation             
4  Jun 22 2022 11:19AM  240179        10         ISDIN Corporation             
5  Jun 22 2022 10:58AM  240177        20  ACI Healthcare USA, Inc.             
6  Jun 22 2022 10:49AM  240175        10          Eywa Pharma Inc.             
7  Jun 22 2022 10:37AM  240174        19    GCH Granules USA, Inc.             
8  Jun 22 2022 10:32AM  240172        10         ISDIN Corporation             
9  Jun 22 2022 10:31AM  240173        10         ISDIN Corporation             

  Executing Released  
0                  1  
1                  2  
2                  7  
3                 32  
4                  8  
5                  1  
6                  6  
7                  1  
8                 38  
9                  7

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                  Customer Released  \
0  Jun 22 2022 11:26AM  240182        10          Methapharm, Inc.        1   
1  Jun 22 2022 11:26AM  240181        10          Methapharm, Inc.        2   
2  Jun 22 2022 11:26AM  240178        10          Methapharm, Inc.        7   
3  Jun 22 2022 11:22AM  240180        10         ISDIN Corporation       32   
4  Jun 22 2022 11:19AM  240179        10         ISDIN Corporation        8   
5  Jun 22 2022 10:58AM  240177        20  ACI Healthcare USA, Inc.        1   
6  Jun 22 2022 10:49AM  240175        10          Eywa Pharma Inc.        6   
7  Jun 22 2022 10:37AM  240174        19    GCH Granules USA, Inc.        1   
8  Jun 22 2022 10:32AM  240172        10         ISDIN Corporation       38   
9  Jun 22 2022 10:31AM  240173        10         ISDIN Corporation        7   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse           Customer Released  \
0  Jun 22 2022 11:26AM  240182        10   Methapharm, Inc.        1   
1  Jun 22 2022 11:26AM  240181        10   Methapharm, Inc.        2   
2  Jun 22 2022 11:26AM  240178        10   Methapharm, Inc.        7   
3  Jun 22 2022 11:22AM  240180        10  ISDIN Corporation       32   
4  Jun 22 2022 11:19AM  240179        10  ISDIN Corporation        8   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Jun 22 2022 11:26AM  240182        10   Methapharm, Inc.       1.0   
1  Jun 22 2022 11:26AM  240181        10   Methapharm, Inc.       2.0   
2  Jun 22 2022 11:26AM  240178        10   Methapharm, Inc.       7.0   
3  Jun 22 2022 11:22AM  240180        10  ISDIN Corporation      32.0   
4  Jun 22 2022 11:19AM  240179        10  ISDIN Corporation       8.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Jun 22 2022 11:26AM  240182        10   Methapharm, Inc.       1.0   
1  Jun 22 2022 11:26AM  240181        10   Methapharm, Inc.       2.0   
2  Jun 22 2022 11:26AM  240178        10   Methapharm, Inc.       7.0   
3  Jun 22 2022 11:22AM  240180        10  ISDIN Corporation      32.0   
4  Jun 22 2022 11:19AM  240179        10  ISDIN Corporation       8.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3122164     128.0       21.0        0.0
102         240089       1.0        0.0        0.0
12          240156       1.0        0.0        0.0
15         1200769      98.0        2.0        0.0
16          480275       1.0        5.0        1.0
19         1921088      10.0        7.0        0.0
20          960506       7.0       24.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3122164     128.0       21.0        0.0
1       102   240089       1.0        0.0        0.0
2        12   240156       1.0        0.0        0.0
3        15  1200769      98.0        2.0        0.0
4        16   480275       1.0        5.0        1.0
5        19  1921088      10.0        7.0        0.0
6        20   960506       7.0       24.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     128.0       21.0        0.0
1       102       1.0        0.0        0.0
2        12       1.0        0.0        0.0
3        15      98.0        2.0        0.0
4        16       1.0        5.0        1.0
5        19      10.0        7.0        0.0
6        20       7.0       24.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  128.0
1       102  Released    1.0
2        12  Released    1.0
3        15  Released   98.0
4        16  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10  102   12    15   16    19    20
Status                                           
Completed    0.0  0.0  0.0   0.0  1.0   0.0   0.0
Executing   21.0  0.0  0.0   2.0  5.0   7.0  24.0
Released   128.0  1.0  1.0  98.0  1.0  10.0   7.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10  102   12    15   16    19    20
0          Completed    0.0  0.0  0.0   0.0  1.0   0.0   0.0
1          Executing   21.0  0.0  0.0   2.0  5.0   7.0  24.0
2           Released  128.0  1.0  1.0  98.0  1.0  10.0   7.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10  102   12    15   16    19    20
0  Completed    0.0  0.0  0.0   0.0  1.0   0.0   0.0
1  Executing   21.0  0.0  0.0   2.0  5.0   7.0  24.0
2   Released  128.0  1.0  1.0  98.0  1.0  10.0   7.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()